In [1]:
from utils import change_pwd
change_pwd()

from crafter_description import *
from contrib import crafter

env = crafter.Env(size=(224, 224))
env.reset()
action_space = env.action_space

['None', 'water', 'grass', 'stone', 'path', 'sand', 'tree', 'lava', 'coal', 'iron', 'diamond', 'table', 'furnace', 'player', 'cow', 'zombie', 'skeleton', 'arrow', 'plant']


In [2]:
obs, info = env.reset(True)

In [17]:
info

{'inventory': {'health': 9,
  'food': 9,
  'drink': 9,
  'energy': 9,
  'sapling': 0,
  'wood': 0,
  'stone': 0,
  'coal': 0,
  'iron': 0,
  'diamond': 0,
  'wood_pickaxe': 0,
  'stone_pickaxe': 0,
  'iron_pickaxe': 0,
  'wood_sword': 0,
  'stone_sword': 0,
  'iron_sword': 0},
 'achievements': {'collect_coal': 0,
  'collect_diamond': 0,
  'collect_drink': 0,
  'collect_iron': 0,
  'collect_sapling': 0,
  'collect_stone': 0,
  'collect_wood': 0,
  'defeat_skeleton': 0,
  'defeat_zombie': 0,
  'eat_cow': 0,
  'eat_plant': 0,
  'make_iron_pickaxe': 0,
  'make_iron_sword': 0,
  'make_stone_pickaxe': 0,
  'make_stone_sword': 0,
  'make_wood_pickaxe': 0,
  'make_wood_sword': 0,
  'place_furnace': 0,
  'place_plant': 0,
  'place_stone': 0,
  'place_table': 0,
  'wake_up': 0},
 'discount': 1.0,
 'semantic': array([[6, 2, 2, ..., 2, 1, 1],
        [2, 2, 2, ..., 1, 1, 1],
        [2, 2, 6, ..., 1, 1, 1],
        ...,
        [2, 2, 2, ..., 1, 1, 1],
        [2, 6, 3, ..., 1, 1, 1],
        [2, 

In [23]:
env.step(2)

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 0.0,
 False,
 {'inventory': {'health': 9,
   'food': 9,
   'drink': 9,
   'energy': 9,
   'sapling': 0,
   'wood': 0,
   'stone': 0,
   'coal': 0,
  

In [5]:
list(info["achievements"].keys())

['collect_coal',
 'collect_diamond',
 'collect_drink',
 'collect_iron',
 'collect_sapling',
 'collect_stone',
 'collect_wood',
 'defeat_skeleton',
 'defeat_zombie',
 'eat_cow',
 'eat_plant',
 'make_iron_pickaxe',
 'make_iron_sword',
 'make_stone_pickaxe',
 'make_stone_sword',
 'make_wood_pickaxe',
 'make_wood_sword',
 'place_furnace',
 'place_plant',
 'place_stone',
 'place_table',
 'wake_up']

In [3]:
describe_env(info)

'Observation (1-step):\n- grass: west\n- grass: east\n- grass: north\n- grass: south (facing)\n\nObservation (nearby):\n- sand 7 steps to your north-west\n- tree 6 steps to your north-east\n- tree 6 steps to your north-east\n- cow 5 steps to your south-east'

In [3]:
def pad_and_slice_semantic_map(info):
    view_height, view_width = info["view"]
    map_height, map_width = info["semantic"].shape
    player_y, player_x = info["player_pos"]
    
    # Calculate padding sizes
    pad_top = pad_bottom = view_height // 2
    pad_left = pad_right = view_width // 2
    
    # Create a padded map with default values (e.g., -1)
    padded_map = np.full((map_height + pad_top + pad_bottom, map_width + pad_left + pad_right), fill_value=-1, dtype=info["semantic"].dtype)
    
    # Place the original semantic map into the padded map
    padded_map[pad_top:pad_top+map_height, pad_left:pad_left+map_width] = info["semantic"]
    
    # Calculate the slicing indexes, taking care to not exceed the padded map bounds
    start_y = player_y + pad_top - view_height // 2
    start_x = player_x + pad_left - view_width // 2
    
    # Slice the padded map to get the view
    semantic_view = padded_map[start_y:start_y+view_height, start_x:start_x+view_width]
    
    return semantic_view

In [4]:
def pad_and_slice_semantic_map(info):
    view_height, view_width = info["view"]  # Desired view size (9x7)
    map_height, map_width = info["semantic"].shape  # Original map size
    player_y, player_x = info["player_pos"]  # Player's position (y, x)

    # Calculate padding sizes to ensure the view can be centered around the player
    pad_top = pad_bottom = view_height // 2
    pad_left = pad_right = view_width // 2

    # Create a padded map with default values (e.g., -1) around the original semantic map
    padded_map = np.full(
        (map_height + pad_top + pad_bottom, map_width + pad_left + pad_right),
        fill_value=-1,  # Use a distinct value to indicate out-of-bounds areas
        dtype=info["semantic"].dtype
    )

    # Insert the original semantic map into the center of the padded map
    padded_map[pad_top:pad_top + map_height, pad_left:pad_left + map_width] = info["semantic"]

    # Calculate the start coordinates for slicing the padded map, ensuring we don't go out of bounds
    start_y = info["player_pos"][0] - info["view"][0] // 2 + pad_top
    end_y = info["player_pos"][0] + info["view"][0] // 2 + 1 + pad_top

    start_x = info["player_pos"][1] - info["view"][1] // 2 + 1 
    end_x = info["player_pos"][1] + info["view"][1] // 2
    

    # Slice the padded map to get the view centered around the player's position
    semantic_view = padded_map[start_y:end_y, start_x:end_x]

    return semantic_view

In [4]:
semantic = pad_and_slice_semantic_map(info)
semantic.shape

(9, 9)

In [26]:
info["player_pos"] = np.array([32, 32])

In [24]:
semantic.shape

(9, 7)

In [5]:
semantic = info["semantic"][
        info["player_pos"][0]
        - info["view"][0] // 2 : info["player_pos"][0]
        + info["view"][0] // 2
        + 1,
        info["player_pos"][1]
        - info["view"][1] // 2
        + 1 : info["player_pos"][1]
        + info["view"][1] // 2,
    ]
center = np.array([info["view"][0] // 2, info["view"][1] // 2 - 1])
x = np.arange(semantic.shape[1])
y = np.arange(semantic.shape[0])
x1, y1 = np.meshgrid(x, y)
semantic.shape

(9, 7)

In [12]:
info

{'distance': 4, 'position': (5, 0)}

In [28]:
semantic.shape

(9, 7)

In [6]:
import numpy as np

def find_closest_blocks(semantic_map):
    id_to_item = ['None', 'water', 'grass', 'stone', 'path', 'sand', 'tree', 'lava', 'coal', 'iron', 'diamond', 'table', 'furnace', 'player', 'cow', 'zombie', 'skeleton', 'arrow', 'plant']
    agent_position = (3, 4)  # Center of a 9x7 map
    closest_positions = {item: {'distance': float('inf'), 'relative_position': None} for item in id_to_item}

    for y in range(semantic_map.shape[0]):
        for x in range(semantic_map.shape[1]):
            block_id = semantic_map[y, x]
            block_name = id_to_item[block_id]
            distance = abs(agent_position[0] - x) + abs(agent_position[1] - y)

            if distance < closest_positions[block_name]['distance']:
                closest_positions[block_name]['distance'] = distance
                # Store the relative position instead of the absolute position
                closest_positions[block_name]['relative_position'] = (x - agent_position[0], y - agent_position[1])

    # Remove entries for blocks not found and 'None' type
    closest_positions = {k: v for k, v in closest_positions.items() if v['relative_position'] is not None and k != 'None'}
    
    return closest_positions

# Example usage with a mock semantic map

closest_blocks = find_closest_blocks(semantic)

for block, info in closest_blocks.items():
    print(f"{block}: Position={info['relative_position']}, Distance={info['distance']}")

grass: Position=(0, -1), Distance=1
player: Position=(0, 0), Distance=0
cow: Position=(3, -4), Distance=7


In [12]:
closest_blocks_old = closest_blocks

In [7]:
result = ""
for xi, yi in zip(x1.flatten(), y1.flatten()):
    result += f" {id_to_item[semantic[yi, xi]]} "
    if xi == semantic.shape[1] - 1:
        result += "\n"
print(result)

 grass  grass  grass  grass  grass  grass  cow 
 grass  grass  grass  grass  grass  grass  grass 
 grass  grass  grass  grass  grass  grass  grass 
 grass  grass  grass  grass  grass  grass  grass 
 grass  grass  grass  player  grass  grass  grass 
 grass  grass  grass  grass  grass  grass  grass 
 grass  grass  grass  grass  grass  grass  grass 
 grass  grass  grass  grass  grass  grass  grass 
 grass  grass  grass  grass  grass  grass  grass 



In [8]:
stuff = env.step(2)

In [10]:
info = stuff[-1]

In [11]:
semantic = pad_and_slice_semantic_map(info)

In [6]:
import numpy as np

# Assume find_closest_blocks is defined as in the previous message, but returns relative positions

def update_relative_positions(closest_blocks, dx, dy):
    """
    Update the relative positions of blocks based on player movement.

    Parameters:
    - closest_blocks: The dictionary of blocks with their relative positions and distances.
    - dx, dy: The change in x and y positions of the player.

    Returns:
    - A new dictionary with updated relative positions.
    """
    updated_blocks = {}
    for block, info in closest_blocks.items():
        # Update the relative position based on player movement
        new_rel_pos = (info['relative_position'][0] - dx, info['relative_position'][1] - dy)
        updated_blocks[block] = {'distance': info['distance'], 'relative_position': new_rel_pos}
    return updated_blocks

def merge_closest_blocks(old_blocks, new_blocks):
    """
    Merge the old and new closest block information, keeping the closest.

    Parameters:
    - old_blocks: The dictionary of old blocks with their relative positions and distances.
    - new_blocks: The dictionary of new blocks with their relative positions and distances.

    Returns:
    - A merged dictionary with the closest block information.
    """
    # Copy old blocks to start with
    merged_blocks = old_blocks.copy()
    for block, new_info in new_blocks.items():
        if block not in merged_blocks or new_info['distance'] < merged_blocks[block]['distance']:
            merged_blocks[block] = new_info  # Update if closer in the new map
    return merged_blocks

# Example of how to use these functions

# Assuming `old_semantic_map` is your initial semantic map and `semantic` is your new semantic map after movement
# closest_blocks_old = find_closest_blocks(old_semantic_map)

# Player movement
dx, dy = 1,0 # Player moves to the right

# Update old relative positions based on player movement
closest_blocks_old_updated = update_relative_positions(closest_blocks_old, dx, dy)

# Find closest blocks in the new map
closest_blocks_new = find_closest_blocks(semantic)

# Merge the old updated and new closest blocks information
merged_closest_blocks = merge_closest_blocks(closest_blocks_old_updated, closest_blocks_new)


(9, 7)

In [7]:
center

array([4, 3])

In [1]:
import matplotlib.pyplot as plt
plt.imshow(env.render())

NameError: name 'env' is not defined

In [2]:
np.unique(info["semantic"])

NameError: name 'np' is not defined

In [10]:
info["semantic"]

array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       ...,
       [4, 3, 3, ..., 1, 1, 1],
       [4, 3, 3, ..., 1, 1, 5],
       [3, 3, 3, ..., 1, 1, 5]], dtype=uint8)

In [11]:
from crafter import make_base_env

env = make_base_env()

/home/renos/rl/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. If you installed TorchRL from PyPI, please report the bug on TorchRL github. If you installed TorchRL locally and/or in development mode, check that you have all the required compiling packages.
  warnings.warn(EXTENSION_WARNING)
/home/renos/anaconda3/envs/torchrl2/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/home/renos/anaconda3/envs/torchrl2/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.

In [12]:
env.observation_spec

CompositeSpec(
    pixels: BoundedTensorSpec(
        shape=torch.Size([224, 224, 3]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([224, 224, 3]), device=cpu, dtype=torch.float32, contiguous=True),
            high=Tensor(shape=torch.Size([224, 224, 3]), device=cpu, dtype=torch.float32, contiguous=True)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous),
    semantic: BoundedTensorSpec(
        shape=torch.Size([1, 9, 7]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([1, 9, 7]), device=cpu, dtype=torch.float32, contiguous=True),
            high=Tensor(shape=torch.Size([1, 9, 7]), device=cpu, dtype=torch.float32, contiguous=True)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous), device=cpu, shape=torch.Size([]))

In [13]:
env.observation_spec

CompositeSpec(
    pixels: BoundedTensorSpec(
        shape=torch.Size([224, 224, 3]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([224, 224, 3]), device=cpu, dtype=torch.float32, contiguous=True),
            high=Tensor(shape=torch.Size([224, 224, 3]), device=cpu, dtype=torch.float32, contiguous=True)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous),
    semantic: BoundedTensorSpec(
        shape=torch.Size([1, 9, 7]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([1, 9, 7]), device=cpu, dtype=torch.float32, contiguous=True),
            high=Tensor(shape=torch.Size([1, 9, 7]), device=cpu, dtype=torch.float32, contiguous=True)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous), device=cpu, shape=torch.Size([]))

In [14]:
env.reset()
tensordict = env.rollout(max_steps=20, policy=None)

In [2]:
from crafter import make_parallel_env_impala

env = make_parallel_env_impala("anything","all",1, device='cpu')

/home/renos/rl/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. If you installed TorchRL from PyPI, please report the bug on TorchRL github. If you installed TorchRL locally and/or in development mode, check that you have all the required compiling packages.
  warnings.warn(EXTENSION_WARNING)
/home/renos/anaconda3/envs/torchrl2/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/home/renos/anaconda3/envs/torchrl2/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.

In [3]:
env.full_observation_spec

CompositeSpec(
    pixels: BoundedTensorSpec(
        shape=torch.Size([1, 3, 64, 64]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([1, 3, 64, 64]), device=cpu, dtype=torch.float32, contiguous=False),
            high=Tensor(shape=torch.Size([1, 3, 64, 64]), device=cpu, dtype=torch.float32, contiguous=False)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous),
    visual_concepts: BoundedTensorSpec(
        shape=torch.Size([1, 19, 9, 9]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([1, 19, 9, 9]), device=cpu, dtype=torch.float32, contiguous=True),
            high=Tensor(shape=torch.Size([1, 19, 9, 9]), device=cpu, dtype=torch.float32, contiguous=True)),
        device=cpu,
        dtype=torch.float32,
        domain=continuous),
    concepts: BoundedTensorSpec(
        shape=torch.Size([1, 1, 26]),
        space=ContinuousBox(
            low=Tensor(shape=torch.Size([1, 1, 26]), device=cpu, dtype=torch.float3

In [4]:
env.reset()
tensordict = env.rollout(max_steps=1000, policy=None)

In [8]:
tensordict["next", "original_reward"].shape

torch.Size([1, 186])

In [7]:
tensordict["next", "terminated"].squeeze(-1)

torch.Size([1, 186, 1])

In [9]:
tensordict["next", "original_reward"][tensordict["next", "terminated"].squeeze(-1)]

tensor([-0.2000])

In [ ]:
tensordict["next", "achievements"][tensordict["next", "terminated"]]

tensor([[0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         2., 0., 0., 2.]])

In [3]:
achievements_complete = torch.tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 2.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 4.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 2.],
        [0., 0., 0., 0., 3., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 3.],
        [0., 0., 0., 0., 2., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 2.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1.],
        [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 3.],
        [0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         2., 0., 0., 2.]])

In [25]:
def calculate_success_rate(achievements_complete):
    achievements_bool = achievements_complete >= 1
    # Calculate mean along the tensor, converting True/False to 1s and 0s implicitly
    success_rate = 100.0 * achievements_bool.float().mean(dim=0)
    return success_rate

success_rate = calculate_success_rate(achievements_complete)

In [26]:
scores = torch.exp(torch.nanmean(np.log(1 + success_rate), -1)) - 1

In [28]:
success_rate

tensor([ 0.0000,  0.0000,  0.0000,  0.0000, 77.7778,  0.0000, 33.3333,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000, 66.6667,  0.0000,  0.0000, 88.8889])

In [27]:
scores

tensor(1.1282)

In [ ]:
['collect_coal',
 'collect_diamond',
 'collect_drink',
 'collect_iron',
 'collect_sapling',
 'collect_stone',
 'collect_wood',
 'defeat_skeleton',
 'defeat_zombie',
 'eat_cow',
 'eat_plant',
 'make_iron_pickaxe',
 'make_iron_sword',
 'make_stone_pickaxe',
 'make_stone_sword',
 'make_wood_pickaxe',
 'make_wood_sword',
 'place_furnace',
 'place_plant',
 'place_stone',
 'place_table',
 'wake_up']

In [20]:
tensordict["pixels"][0,0].max()

tensor(0.4999)

In [8]:
tensordict["pixels"].shape

torch.Size([1, 20, 4, 84, 84])

In [1]:
import matplotlib.pyplot as plt
plt.imshow(tensordict["pixels"][0,1,3].reshape((84, 84 ,1)) / 255.)

NameError: name 'tensordict' is not defined

In [19]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
import random
action_space = env.action_space
# Create a figure for plotting
fig, ax = plt.subplots()

# Function to update the plot with each frame
def update(frame):
    action = random.randint(0, action_space.n - 1)
    obs, reward, done, info = env.step(action)
    ax.clear()  # Clear previous frame
    ax.axis('off')  # Hide axes
    ax.imshow(env.render())  # Show current observation
    return ax,

# Create an animation
ani = FuncAnimation(fig, update, frames=range(10), blit=False)

plt.close(fig)  # Prevent displaying a static plot

# Save the animation as an MP4 file
ani.save('/home/renos/rl-concepts/env_animation_frames_before.mp4', writer='ffmpeg', dpi=80, fps=30)

AttributeError: 'list' object has no attribute 'n'

In [ ]:
import pickle

with open('/home/renos/rl-concepts/env.pkl', 'wb') as f:
    pickle.dump(env, f)

In [ ]:
import pickle 

with open('/home/renos/rl-concepts/env.pkl', 'rb') as f:
    env = pickle.load(f)

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
import random
action_space = env.action_space
# Create a figure for plotting
fig, ax = plt.subplots()

# Function to update the plot with each frame
def update(frame):
    action = random.randint(0, action_space.n - 1)
    obs, reward, done, info = env.step(action)
    ax.clear()  # Clear previous frame
    ax.axis('off')  # Hide axes
    ax.imshow(env.render())  # Show current observation
    return ax,

# Create an animation
ani = FuncAnimation(fig, update, frames=range(100), blit=False)

plt.close(fig)  # Prevent displaying a static plot

# Save the animation as an MP4 file
ani.save('/home/renos/rl-concepts/env_animation_frames_after.mp4', writer='ffmpeg', dpi=80, fps=30)


In [1]:
import pickle

#open skill_library.pkl file
with open('/home/renos/rl-concepts/skill_library.pkl', 'rb') as f:
    skill_library = pickle.load(f)

In [2]:
skill_library

{'CollectResource': {'skill_desc': 'Gather specified resources from the environment.',
  'skill_param': {'resource_type': 'stone'},
  'skill_guide': "Use the 'Do' action while facing a stone resource with the appropriate tool (wood pickaxe) to collect stone."},
 'PlaceFurniture': {'skill_desc': 'Place a piece of furniture in the environment.',
  'skill_param': 'type of furniture',
  'skill_guide': "Identify a suitable location within the environment that is clear of obstacles. Use the 'Place' action with the specified furniture type to place it in the chosen location."},
 'CraftItem': {'skill_desc': 'Use available resources to craft a specified item at a crafting station.',
  'skill_param': {'item_to_craft': 'The specific item you want to craft.',
   'required_resources': 'The resources needed to craft the item.',
   'crafting_station': 'The location or type of crafting station required.'},
  'skill_guide': 'To craft an item, ensure you have all the required resources in your inventory